# Building and Deploying Deep Learning Applications with TensorFlow

In [ ]:
# TensorFlow Machine Learning Models
# Image Recognition
# Speech

In [2]:
import os
import tensorflow as tf

/home/hamza/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')
addition = tf.add(X,Y, name='addition')

In [5]:
with tf.Session() as sess:
    result = sess.run(addition, feed_dict={X: [1,2,10], Y: [4,2,10]})
    print(result)

[ 5.  4. 20.]


In [6]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [7]:
training_data_df = pd.read_csv('../Datasets/Ex_Files_TensorFlow/Exercise Files/03/sales_data_training.csv', dtype=float)

X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']].values

In [8]:
testing_data_df = pd.read_csv('../Datasets/Ex_Files_TensorFlow/Exercise Files/03/sales_data_test.csv', dtype=float)

X_testing = testing_data_df.drop('total_earnings', axis=1).values
Y_testing = testing_data_df[['total_earnings']].values

In [9]:
X_scaler = MinMaxScaler(feature_range=(0,1))
Y_scaler = MinMaxScaler(feature_range=(0,1))

X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

In [10]:
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

In [11]:
#Training the TensorFlow model

In [12]:
learning_rate = 0.001
training_epochs = 100
display_step = 5

number_of_inputs = 9
number_of_outputs = 1

layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 1

In [13]:
#input layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))

In [14]:
#Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable(name='weights1', shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name='biases1', shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_outputs = tf.nn.relu(tf.matmul(X, weights) + biases)
    

In [15]:
#Layer 2

with tf.variable_scope('layer_2'):
    weights = tf.get_variable(name='weights2', shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name='biases2', shape=[layer_2_nodes], initializer=tf.zeros_initializer)
    layer_2_outputs = tf.nn.relu(tf.matmul(layer_1_outputs, weights) + biases)
    

In [16]:
#Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name='weights3', shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name='biases3', shape=[layer_3_nodes], initializer=tf.zeros_initializer)
    layer_3_outputs = tf.nn.relu(tf.matmul(layer_2_outputs, weights) + biases)
    
    

In [17]:
#Layer 4
with tf.variable_scope('layer_4'):
    weights = tf.get_variable(name='weights4', shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name='biases4', shape=[number_of_outputs], initializer=tf.zeros_initializer)
    prediction = tf.nn.relu(tf.matmul(layer_3_outputs, weights) + biases)
    

In [18]:
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

In [19]:
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    
    

In [20]:
saver = tf.train.Saver()

In [33]:
with tf.variable_scope('loggin'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()
    
    
with tf.Session() as sess:
    
#     saver.restore(sessm 'logs/train_model.ckpt')
    sess.run(tf.global_variables_initializer())
    
    training_writer = tf.summary.FileWriter('./logs/training', sess.graph)
    testing_writer = tf.summary.FileWriter('./logs/testing', sess.graph)
    
    for epoch in range(training_epochs):
        sess.run(optimizer, feed_dict={X:X_scaled_training, Y:Y_scaled_training})
        if epoch % 5 == 0:
            training_cost, training_summary = sess.run([cost, summary],feed_dict={X:X_scaled_training, Y:Y_scaled_training})
            testing_cost, testing_summary = sess.run([cost,summary],feed_dict={X:X_scaled_testing, Y:Y_scaled_testing})
            print(epoch, training_cost, testing_cost)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)
            
#         print("Training pass: {}".format(epoch))

    print("Training Completed")
    training_cost = sess.run(cost,feed_dict={X:X_scaled_training, Y:Y_scaled_training})
    testing_cost = sess.run(cost,feed_dict={X:X_scaled_testing, Y:Y_scaled_testing})    
    print("Final Testing Cost: {}".format(testing_cost))
    save_path = saver.save(sess, "logs/train_model.ckpt")
    print(save_path)
    
    
    model_builder = tf.saved_model.builder.SavedModelBuilder('exported_model')

    inputs = {'inputs': tf.saved_model.utils.build_tensor_info(X)}
    outputs = {'earnings': tf.saved_model.utils.build_tensor_info(prediction)}

    signature_def = tf.saved_model.signature_def_utils.build_signature_def(
        inputs=inputs,
        outputs=outputs,
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
    )

    model_builder.add_meta_graph_and_variables(
        sess, 
        tags=[tf.saved_model.tag_constants.SERVING],
        signature_def_map={tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY}
    )
    

0 0.0862571 0.0938248
5 0.020053478 0.020261955
10 0.0267303 0.026801635
15 0.008975815 0.008903066
20 0.00858576 0.008717244
25 0.007794181 0.007693882
30 0.0045589544 0.004162862
35 0.004959629 0.004570704
40 0.0032362342 0.002962879
45 0.0030110292 0.0029125556
50 0.0023903523 0.002335669
55 0.0020763383 0.0020457716
60 0.0017542728 0.0017563774
65 0.001525028 0.0015572361
70 0.0013289561 0.0013640919
75 0.0011933162 0.0012349004
80 0.001052668 0.0011187037
85 0.00095251 0.0010437462
90 0.0008569539 0.000971376
95 0.00078425475 0.000914687
Training Completed
Final Testing Cost: 0.0008651260868646204
logs/train_model.ckpt


TypeError: 'set' object is not subscriptable

AssertionError: Export directory already exists. Please specify a different export directory: exported_model

In [ ]:
    !tensorboard --logdir='./logs/training'